In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Read the csv file into pandas Dataframe
credit_data = pd.read_csv('Resources/credit_data.csv')
credit_data.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [3]:
## Basic formatting for ease of use/prevent errors

# Lowercase for all headings
credit_data.columns = credit_data.columns.str.lower()
credit_data.columns

Index(['id', 'customer_id', 'month', 'name', 'age', 'ssn', 'occupation',
       'annual_income', 'monthly_inhand_salary', 'num_bank_accounts',
       'num_credit_card', 'interest_rate', 'num_of_loan', 'type_of_loan',
       'delay_from_due_date', 'num_of_delayed_payment', 'changed_credit_limit',
       'num_credit_inquiries', 'credit_mix', 'outstanding_debt',
       'credit_utilization_ratio', 'credit_history_age',
       'payment_of_min_amount', 'total_emi_per_month',
       'amount_invested_monthly', 'payment_behaviour', 'monthly_balance',
       'credit_score'],
      dtype='object')

In [4]:
## Get an understanding the data
# credit_data.info()

In [5]:
## Determine if duplicates in data
# check on SSN identifier

In [6]:
## Drop columns that aren't necessary

# 'ID' is persons ID, arbitrary, not predictive in credit scoring, not required
# 'Customer_ID' as above
# 'Name' as above
# 'SSN' as above

credit_data_1 = credit_data.drop(columns=["id", "customer_id", "name", "ssn"])
credit_data_1.head()

,month,age,occupation,annual_income,monthly_inhand_salary,num_bank_accounts,num_credit_card,interest_rate,num_of_loan,type_of_loan,...,credit_mix,outstanding_debt,credit_utilization_ratio,credit_history_age,payment_of_min_amount,total_emi_per_month,amount_invested_monthly,payment_behaviour,monthly_balance,credit_score
0,January,23,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,February,23,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,March,-500,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,April,23,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,May,23,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


### Preprocessing

##### Working through each columns to understand anomalous values, and alter or delete as appropriate

In [7]:
## AGE

# Understand the range of ages, and if age values make sense
credit_data_1["age"].value_counts().sort_index().head(20)

# From perusal of column data, can see many values include'_'.
# Count how many rows with an underscore
US_credit_data_1 = credit_data_1["age"].str.contains('_').sum()
print(f"{US_credit_data_1} out of 100k rows, have a string, 4%")

# Remove underscore '_'
# credit_data_1.loc[credit_data_1["age"] == "_", "age"] = ""
    # code for where value is '_' only, not part of value is '_'
# Code to replace '_' with ""
credit_data_1["age"] = credit_data_1["age"].str.replace("_", "")

# Try changing all values to numeric. Will prove no more anomalous values
credit_data_1["age"] = pd.to_numeric(credit_data_1["age"])
f"Able to convert all values to numeric, suggesting, no more anomalous values"


4939 out of 100k rows, have a string, 4%


'Able to convert all values to numeric, suggesting, no more anomalous values'

In [8]:
## Many ages > 85 and <14 (assuming 14 is universal age ppl are allowed to be officially employed)
# Count how many values are <18 and >90, as unlikely to be seeking loans.
# If insignificant, can delete rows
credit_data_2 = credit_data_1.loc[(credit_data_1["age"] < 14) | (credit_data_1["age"] > 85)]
print(credit_data_2["age"].describe())
length_18_85 = len(credit_data_2["age"])
f"{length_18_85} of 100k values are <14 & >85. Will be deleted"

# DF updated to exclude <14 and >85
credit_data_3 = credit_data_1.loc[(credit_data_1["age"] > 14) & (credit_data_1["age"] < 85)]


count    2781.000000
mean     2813.963323
std      3068.568546
min      -500.000000
25%      -500.000000
50%      2318.000000
75%      5587.000000
max      8698.000000
Name: age, dtype: float64


In [9]:
## MONTH

credit_data_3["month"].value_counts()
# Data included from Jan - July.
# No anomalous entries.
# Data OK

August      12094
July        12059
June        12036
May         12018
March       12002
April       11978
February    11935
January     11922
Name: month, dtype: int64

In [10]:
## OCCUPATION

credit_data_3["occupation"].value_counts()

# credit_data_1["occupation"].value_counts().sum()
# 7% of ppl don't have a recorded occupation.

# Will either convert "_______" (as copied from results below) Other.
credit_data_3.loc[credit_data_3["occupation"] == "_______", "occupation"] = "Other"
# credit_data_3

In [11]:
## ANNUAL INCOME

credit_data_3["annual_income"].value_counts()
# There are strings of '_' in income. Need to delete, use code as above.
credit_data_3["annual_income"] = credit_data_3["annual_income"].str.replace("_", "")
# credit_data_3["annual_income"].describe()
# Count matches DF length, data OK.


In [12]:
## MONTHLY INCOME

credit_data_3["monthly_inhand_salary"].value_counts()
# Data OK

2295.058333     15
6769.130000     15
6082.187500     15
3080.555000     14
6358.956667     14
                ..
10085.577500     1
4237.487520      1
1387.232104      1
2514.590000      1
835.827500       1
Name: monthly_inhand_salary, Length: 13124, dtype: int64

In [13]:
## NUM OF BANK ACCS

credit_data_3["num_bank_accounts"].value_counts()
# Shows values in the 1000s, intuitively wrong, unlikely for a person to have 1000 bank accounts.

# Count number of values with banks accounuts > than 20
bank_acc_20plus = credit_data_3.loc[(credit_data_3["num_bank_accounts"] > 20)]
print(bank_acc_20plus["num_bank_accounts"].describe())
f"1274 of 100k values, will be deleted"

credit_data_4 = credit_data_3.loc[(credit_data_3["num_bank_accounts"] < 20)]


count    1274.000000
mean      899.966248
std       515.214421
min        26.000000
25%       464.000000
50%       889.000000
75%      1343.500000
max      1798.000000
Name: num_bank_accounts, dtype: float64


In [14]:
## NUM OF CREDIT CARDS

credit_data_4["num_credit_card"].value_counts()

# Shows values in the 100s, intuitively wrong, unlikely for a person to have 100s of credit cards.

# Count number of values with banks accounuts > than 10
cc_10plus = credit_data_4.loc[(credit_data_4["num_credit_card"] > 10)]
print(cc_10plus["num_credit_card"].describe())
f"2194 of 100k values, will be deleted"

credit_data_5 = credit_data_4.loc[(credit_data_4["num_credit_card"] <= 10)]

count    2194.000000
mean      737.804011
std       440.830082
min        11.000000
25%       336.250000
50%       740.500000
75%      1123.000000
max      1499.000000
Name: num_credit_card, dtype: float64


In [15]:
## INTEREST_RATE

credit_data_5["interest_rate"].value_counts()

# Shows values in the 100s, unlikely that interest rates would be >50% on credit cards.

# Count number of values with banks accounuts > than 10
ir_30plus = credit_data_5.loc[(credit_data_5["interest_rate"] > 50)]
print(ir_30plus["interest_rate"].describe())
f"1899 of 100k values, will be deleted"

credit_data_6 = credit_data_5.loc[(credit_data_5["interest_rate"] <= 50)]

count    1899.000000
mean     2872.620853
std      1661.728430
min        59.000000
25%      1387.000000
50%      2851.000000
75%      4301.000000
max      5797.000000
Name: interest_rate, dtype: float64


In [16]:
## NUM_OF_LOAN

credit_data_6["num_of_loan"].value_counts()
credit_data_6["num_of_loan"].describe()
credit_data_6["num_of_loan"].min()
# Count number of values < 0
# loan_less0 = credit_data_6.loc[(credit_data_6["num_of_loan"] < 0)]
# Above code has strings in it. Assume its same string as before '_', delete this string.
credit_data_6["num_of_loan"] = credit_data_6["num_of_loan"].str.replace("_", "")

# Convert data to numeric
credit_data_6["num_of_loan"] = pd.to_numeric(credit_data_6["num_of_loan"])


In [30]:
loan_less0 = credit_data_6.loc[(credit_data_6["num_of_loan"] < 0)]
# loan_less0["num_of_loan"].describe()
f"3512 of 100k values, will be deleted"

credit_data_7 = credit_data_6.loc[(credit_data_6["interest_rate"] >= 0)]

In [45]:
## TYPE OF LOAN

credit_data_7["type_of_loan"].value_counts()
# Shows there are many values with nil, due to person having 0 loans.
# Replace blank values, with 'None'
latest = credit_data_7["type_of_loan"].fillna("None", inplace = True)



In [49]:
## DELAY FROM DUE DATE

credit_data_7["delay_from_due_date"].value_counts()
# no anomalous figures. -ve values represent payments in advance.

 15    3265
 13    3147
 8     3058
 14    3009
 10    3006
       ... 
 64      55
 65      52
-5       29
 66      29
 67      17
Name: delay_from_due_date, Length: 73, dtype: int64

In [51]:
## NUM OF DELAYED PAYMENT

credit_data_7["num_of_delayed_payment"].value_counts()
credit_data_7["num_of_delayed_payment"].describe()

# From perusal, there are erroneous strings, remove
credit_data_7["num_of_delayed_payment"] = credit_data_7["num_of_delayed_payment"].str.replace("_", "")

# Replace blank values with 0 integer, to ability to calculate as a factor later
credit_data_7["num_of_delayed_payment"] = credit_data_7["num_of_delayed_payment"].fillna(0)


In [53]:
## CHANGED CREDIT LIMIT

credit_data_7["changed_credit_limit"].value_counts()
credit_data_7["changed_credit_limit"].describe()

# From perusal, there are erroneous strings, remove
credit_data_7["changed_credit_limit"] = credit_data_7["changed_credit_limit"].str.replace("_", "")

# Replace blank values with 0 integer, to ability to calculate as a factor later
credit_data_7["changed_credit_limit"] = credit_data_7["changed_credit_limit"].fillna(0)

In [59]:
## NUM OF CREDIT INQUIRIES

credit_data_7["num_credit_inquiries"].value_counts()
credit_data_7["num_credit_inquiries"].describe()

# Replace blank values with 0 integer, to ability to calculate as a factor later
credit_data_7["num_credit_inquiries"] = credit_data_7["num_credit_inquiries"].fillna(0)

# Len same as DF whole, suggesting, all records (rows) have a value.

In [63]:
## CREDTI MIX

credit_data_7["credit_mix"].value_counts()
# credit_data_7["credit_mix"].describe()

# Replace blank values with 0 integer, to ability to calculate as a factor later
# credit_data_7["credit_mix"] = credit_data_7["credit_mix"].fillna(0)

# From value_counts, 1 category is '_', replace with 'None'
credit_data_7["credit_mix"] = credit_data_7["credit_mix"].str.replace("_", "None")

In [69]:
## OUTSTANDING DEBT

credit_data_7["outstanding_debt"].value_counts()
# credit_data_7["credit_mix"].describe()

# From perusal, there are erroneous strings, remove
credit_data_7["outstanding_debt"] = credit_data_7["outstanding_debt"].str.replace("_", "")

# Convert column to numeric
credit_data_7["outstanding_debt"] = pd.to_numeric(credit_data_7["outstanding_debt"])


In [71]:
## CREDIT UTILIZATION RATIO

credit_data_7["credit_utilization_ratio"].value_counts()

# Convert column to numeric
credit_data_7["credit_utilization_ratio"] = pd.to_numeric(credit_data_7["credit_utilization_ratio"])


In [81]:
## CREDIT HISTORY AGE

credit_data_7["credit_history_age"].value_counts()

# Replace blank values with 0 integer, to ability to calculate as a factor later
credit_data_7["credit_history_age"] = credit_data_7["credit_history_age"].fillna("0")

In [82]:
credit_data_7[["year", "ystr", "andstr", "month", "mstr"]] = credit_data_7.credit_history_age.str.split(" ", expand = True)

In [83]:
credit_data_7[["year", "ystr", "andstr", "month", "mstr"]]

,year,ystr,andstr,month,mstr
0,22,Years,and,1,Months
1,None,None,None,None,None
3,22,Years,and,4,Months
4,22,Years,and,5,Months
5,22,Years,and,6,Months
...,...,...,...,...,...
99994,31,Years,and,5,Months
99995,31,Years,and,6,Months
99996,31,Years,and,7,Months
99998,31,Years,and,9,Months
